# Text generator based on RNN

### Import libraries

In [1]:
import tensorflow as tf
import numpy as np

### Configurations

In [2]:
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            "\\^_abcdefghijklmnopqrstuvwxyz{|}")
TestText="Too young, too simple, sometimes naive. "
graphPath=r".\graphs"
num_step=len(TestText)-1
batchSize=1
def Encode(string,name=None):
    tensor=[vocab.find(ch)+1 for ch in string]
    tensor = tf.one_hot(tensor, depth=len(vocab)+1, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32, name=name)
    with tf.Session() as sess:
        nparray=tensor.eval()
    return nparray
def Decode(nparray):
    return "".join([vocab[index-1] if index>0 else "[INVALID]" for index in np.argmax(nparray,axis=1)])
X=Encode(TestText)
Y=X[1:]
X=X[:-1]
print(X.shape,Y.shape)

(39, 84) (39, 84)


### Create graph

In [3]:
class MyRNN():
    def __init__(self,inputVectorDim,stateSize=100,scope="RNN",outputVectorDim=None):
        if outputVectorDim==None:
            outputVectorDim=inputVectorDim
        with tf.variable_scope(scope) as modelScope:
            with tf.variable_scope("structure") as structureScope:
                #One batch at a time
                self.inputs=tf.placeholder(dtype=tf.float32,shape=[None,inputVectorDim])
                inputShape=tf.shape(self.inputs)
                self.outputs=tf.placeholder(dtype=tf.float32,shape=[None,outputVectorDim])
                self.__cell=tf.contrib.rnn.GRUCell(num_units=stateSize)
                #S20:State to output
                self.__wS2O=tf.get_variable(name="Weight_s2o",shape=[stateSize,outputVectorDim],initializer=tf.truncated_normal_initializer())
                self.__bS2O=tf.get_variable(name="Bias_s2o",shape=[1,outputVectorDim],initializer=tf.truncated_normal_initializer())
                #add a "batch dimension" to the front
                cellInputs=tf.reshape(tensor=self.inputs,shape=[1,-1,inputVectorDim])
                outputs,finalState=tf.nn.dynamic_rnn(cell=self.__cell,inputs=cellInputs,dtype=tf.float32)
                outputs = tf.reshape(tensor=outputs, shape=[-1, stateSize])#Remove the batch dimension ((
                self.netOutputs=tf.add(tf.matmul(outputs,self.__wS2O),self.__bS2O)
            with tf.name_scope("training"):
                loss=tf.losses.softmax_cross_entropy(logits=self.netOutputs,onehot_labels=self.outputs)
                loss=tf.reduce_mean(loss,name="loss")
                self.optimizer=tf.train.AdamOptimizer(learning_rate=0.001,name="optimizer").minimize(loss)
            with tf.name_scope("summary") as scope:
                self.summary_op=tf.summary.merge_all()
    def onlineInference(self,intputTensor,timeSteps=100,state=None,seed=None):
        #To be fixed
        tf.one_hot(off_value=0.0,on_value=1.0)
        seq=[]
        input_=seed
        for length in range(timeSteps):
            output,state=tf.nn.dynamic_rnn(cell=self.__cell,initial_state=state,dtype=tf.float32,inputs=input_)
            outputs = tf.reshape(tensor=outputs, shape=[-1, stateSize],)
            seq.append(output)
            

In [4]:
Test=MyRNN(scope="RNN",inputVectorDim=len(vocab)+1)

In [5]:
with tf.Session() as sess:
    writer=tf.summary.FileWriter(logdir=graphPath,graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    result=sess.run(Test.netOutputs,{Test.inputs:X,Test.outputs:Y})
    writer.close()
print(Decode(result))

(((((((((((1((1WW((((WWWW((WW((WW((((((


In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        sess.run(Test.optimizer,feed_dict={Test.inputs:X,Test.outputs:Y})
    result=sess.run(Test.netOutputs,{Test.inputs:X})
print(Decode(result))

oo young, too simple, sometimes naive. 
